In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
import time
llm = ChatOpenAI(
    temperature=0.1
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(separator = "\n", chunk_size = 600, chunk_overlap = 100,)

loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            주어진 문서를 읽고 해석해
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm

def map_docs(input):
    documents = input['documents']
    question = input['question']
    results = []                                                                                                    # 
    for doc in documents:                                                                                           # 
        result = map_doc_chain.invoke({"context": doc.page_content, "question": question}).content                  # 
        results.append(result)                                                                                      #
        time.sleep(20)
    return result

map_chain = {"documents":retriever, "question":RunnablePassthrough(), } | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages([
    ("system","""문서에 대하여 해석하고 추측할 수 있는 부분에 대해서 답변해줘 만약 문서와 전혀 관련이 없는 내용이라면 모른다고 대답해------ {context}""",),
    ("human","{question}"),
])

chain = {"context":map_chain, "question":RunnablePassthrough()} | final_prompt | llm

chain.invoke("등장인물을 알려줘")

AIMessage(content='주어진 문서에서 언급된 두 명의 주요 등장인물은 consult하는 사람과 그의 동료인 Holmes입니다. consult하는 사람은 특별한 문제를 상의하고 싶어하는 신사로서, Holmes는 이 문제를 해결하기 위해 노력하는 탐정입니다.')